In [1]:
import pandas as pd
import requests
from configparser import ConfigParser
import boto3
from botocore.exceptions import NoCredentialsError
import io

In [2]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("./airflow/config/config.ini")

['./airflow/config/config.ini']

In [3]:
url_municipios_ibge = config.get("URL", "municipios_ibge")
url_municipios_ibge

'https://servicodados.ibge.gov.br/api/v1/localidades/municipios'

In [4]:
# Fazer a requisição para obter o JSON da URL
response = requests.get(url_municipios_ibge)
municipios_data = response.json()

In [ ]:
municipios_data

In [5]:
# Carregar o JSON em um DataFrame Pandas
df = pd.json_normalize(municipios_data)

In [ ]:
df.head()

In [9]:
# Configurações para conexão com o MinIO/S3
s3_endpoint = config.get("MinIO", "endpoint")
# s3_endpoint = 'localhost:9001'
s3_access_key = config.get("MinIO", "access_key")
s3_secret_key = config.get("MinIO", "secret_key")
s3_bucket_raw = config.get("Bucket", "bucket_raw")
s3_bucket_context = config.get("Bucket", "bucket_context")
prefix_municipios_ibge_json = config.get("Bucket", "prefix_municipios_ibge_json")
prefix_municipios_ibge_csv = config.get("Bucket", "prefix_municipios_ibge_csv")

In [10]:
print("s3_endpoint = ", s3_endpoint)
print("s3_access_key = ", s3_access_key)
print("s3_secret_key = ", s3_secret_key)
print("s3_bucket_raw = ", s3_bucket_raw)
print("s3_bucket_context = ", s3_bucket_context)
print("prefix_municipios_ibge_json = ", prefix_municipios_ibge_json)
print("prefix_municipios_ibge_csv = ", prefix_municipios_ibge_csv)

s3_endpoint =  http://minio:9000
s3_access_key =  aulafia
s3_secret_key =  aulafia@123
s3_bucket_raw =  raw
s3_bucket_context =  context
prefix_municipios_ibge_json =  municipios_ibge_json/
prefix_municipios_ibge_csv =  municipios_ibge_csv/


In [11]:
# Inicializar o cliente boto3 para S3
s3_client = boto3.client(
    "s3",
    endpoint_url=s3_endpoint,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

In [14]:
# Armazenar o JSON no bucket 'raw'
json_filename = "MunicipiosIBGE.json"
json_s3_path = f"{prefix_municipios_ibge_json}{json_filename}"

# try:
#     s3_client.put_object(Bucket=s3_bucket_raw, Key=json_s3_path, Body=response.content)
#     print(f"JSON file uploaded to MinIO: {json_s3_path}")
# except NoCredentialsError:
#     print("Credentials not available.")

s3_client.put_object(Bucket=s3_bucket_raw, Key=prefix_municipios_ibge_json + json_filename, Body=response.content)

EndpointConnectionError: Could not connect to the endpoint URL: "http://minio:9000/raw/municipios_ibge_json/MunicipiosIBGE.json"

In [ ]:
# Transformar o DataFrame Pandas em CSV em memória
csv_io = io.StringIO()
df.to_csv(csv_io, index=False)
csv_data = csv_io.getvalue()

In [ ]:
print(csv_data)